In [1]:
import bambi as bmb
from bambi.defaults import get_builtin_family

In [2]:
family = get_builtin_family("gaussian")

Cosas que estan bien

* `bmb.Formula`
* `bmb.Link`
* `bmb.Likelihood?`
* `bmb.terms.*`
    * Excepto `bmb.terms.offset.OffsetTerm`
    * Está bien que los priors esten asociados a los términos

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
data = bmb.load_data("my_data")

In [5]:
formula = bmb.Formula("y ~ x", "sigma ~ x")
priors = {
    "y": {
        "x": bmb.Prior("Normal", mu=0, sigma=1)
    },
    "sigma": {
        "Intercept": bmb.Prior("Normal", mu=-1, sigma=5)
    },
}
model = bmb.Model(formula, data, priors=priors)
print(model.distributional_components["y"].terms)
print(model.distributional_components["sigma"].terms)

{'Intercept': CommonTerm(  
  name: Intercept,
  prior: Normal(mu: 0, sigma: 1),
  shape: (50,),
  categorical: False
), 'x': CommonTerm(  
  name: x,
  prior: Normal(mu: 0, sigma: 1),
  shape: (50,),
  categorical: False
)}
{'Intercept': CommonTerm(  
  name: Intercept_sigma,
  prior: Normal(mu: -1, sigma: 5),
  shape: (50,),
  categorical: False
), 'x': CommonTerm(  
  name: x_sigma,
  prior: Normal(mu: 0, sigma: 1),
  shape: (50,),
  categorical: False
)}


In [6]:
formula = bmb.Formula("y ~ x")
priors = {"x": bmb.Prior("Normal", mu=0, sigma=1)}
model = bmb.Model(formula, data, priors=priors)
print(model.distributional_components["y"].terms)

{'Intercept': CommonTerm(  
  name: Intercept,
  prior: Normal(mu: 0, sigma: 1),
  shape: (50,),
  categorical: False
), 'x': CommonTerm(  
  name: x,
  prior: Normal(mu: 0, sigma: 1),
  shape: (50,),
  categorical: False
)}


In [10]:
model.constant_components["sigma"].prior

In [23]:
model.components["y"].__dict__

{'terms': {'Intercept': CommonTerm(  
    name: Intercept,
    prior: None,
    shape: (50,),
    categorical: False
  ),
  'x': CommonTerm(  
    name: x,
    prior: None,
    shape: (50,),
    categorical: False
  )},
 'design': DesignMatrices
 
                   (rows, cols)
 Response:                (50,)
 Common:                (50, 2)
 
 Use .reponse, .common, or .group to access the different members.,
 'response_name': 'y',
 'response_kind': 'data',
 'spec': Model,
 '_suffix': ''}

In [11]:
1 + 1

'mu'

Algunos principios

* Cada formula es un `DistributionalComponent`
    * Si la respuesta es el nombre de una variable, `response_kind = "data"` 
    * Si la respuesta es el nombre de un parametro auxiliar, `response_kind = "parameter"`.

**Como manejar los casos donde los parametros auxiliares son fijos?**

* Esto equivale a modelos de interceptos para estos parametros
    * Hay que lograrlo sin llamar al parametro con el nombre "Intercept_parameter"

A tener en cuenta

* No solo es cambiar la interface de la formula
* Tambien es habilitar diferentes funciones de enlace para diferentes terminos


**Idea 1**

```python
link_dicts = {"mu": "identity", "sigma": "log"}
Model(... link=link_dicts)
```

La pregunta es, donde se almacenan estos `links`?

**Respuesta:** Tienen que ir a parar a las familias.

**Pregunta:** 

* El prior de los parametros auxiliares tiene que ir en el Likelihood???
* Empiezo a pensar que la respuesta es **NO**. Estamos mezclado dos cosas distintas.
    * Descripcion de la funcion de verosimilitud, sus parametros, y cual es el rol de cada uno. -> Esto siempre es necesario
    * Definicion de distribuciones a prior. -> Esto no es siempre necesario.
* Los Priors en la familia tambien son un **error**.
    * Un prior no forma parte de la descripcion de la familia.

TODO

Verificar que tenemos los SUPPORTED_LINKS para cada parametro (no todos deben tolerar todo)

Mapping
* names -> DesignMatrices